In [18]:
import ast
import itertools
import numpy as np
import pandas as pd
import re
import spacy
import torch
from sklearn.feature_extraction.text import TfidfVectorizer


In [19]:
df = pd.read_csv('dataset/movies_metadata.csv', encoding='utf-8')
# df = df.dropna()
df.shape

C:\Users\wsnia\AppData\Local\Temp\ipykernel_10852\2502949224.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset/movies_metadata.csv', encoding='utf-8')


(45466, 24)

In [20]:
# df['budget'] = df['budget'].astype(int)
df['genres'] = df['genres'].apply(ast.literal_eval)
df['genre_names'] = df['genres'].apply(lambda genre_list: [genre['name'] for genre in genre_list])
# df['production_companies'] = df['production_companies'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
# df['production_companies_names'] = df['production_companies'].apply(lambda genre_list: [genre['name'] for genre in genre_list])
# df['production_companies'] = df['production_companies'].apply(ast.literal_eval)
# df['production_companies_names'] = df['production_companies'].apply(
#     lambda genre_list: [genre['name'] for genre in genre_list] if isinstance(genre_list, list) else []
# )
# df['production_companies_names'] = df['production_companies'].apply(lambda genre_list: [genre['name'] if pd.notna(genre_list) else [] for genre in genre_list])
# df['production_countries'] = df['production_countries'].apply(ast.literal_eval)
# df['production_countries_names'] = df['production_countries'].apply(lambda genre_list: [genre['name'] for genre in genre_list])
# df['production_countries'] = df['production_countries'].apply(ast.literal_eval)
# df['production_countries_names'] = df['production_countries'].apply(
    # lambda genre_list: [genre['name'] for genre in genre_list] if isinstance(genre_list, list) else []
# )
# df['release_date'] = pd.to_datetime(df['release_date'])
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')
df_cleaned = df.dropna(subset=['release_date'])
df['release_year'] = df['release_date'].dt.year
df = df.drop(columns=['belongs_to_collection', 'homepage', 'status', 'video', 'imdb_id', 'budget', 'poster_path', 'revenue', 'spoken_languages', 'genres', 'production_companies', 'production_countries', 'release_date' ])
df.rename(columns={'genre_names': 'genres', 'production_companies_names': 'production_companies', 'production_countries_names' : 'production_countries'}, inplace=True)
df.dtypes
df.to_csv('dataset/movies_metadata_preprocessed_big.csv', index=False) 
df

,adult,id,original_language,original_title,overview,popularity,runtime,tagline,title,vote_average,vote_count,genres,release_year
0,False,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,81.0,NaN,Toy Story,7.7,5415.0,"[Animation, Comedy, Family]",1995.0
1,False,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,104.0,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[Adventure, Fantasy, Family]",1995.0
2,False,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,101.0,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,"[Romance, Comedy]",1995.0
3,False,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,127.0,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"[Comedy, Drama, Romance]",1995.0
4,False,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,106.0,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,[Comedy],1995.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,439050,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,90.0,Rising and falling between a man and woman,Subdue,4.0,1.0,"[Drama, Family]",NaN
45462,False,111109,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,360.0,NaN,Century of Birthing,9.0,3.0,[Drama],2011.0
45463,False,67758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,90.0,A deadly game of wits.,Betrayal,3.8,6.0,"[Action, Drama, Thriller]",2003.0
45464,False,227506,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,87.0,NaN,Satan Triumphant,0.0,0.0,[],1917.0


In [21]:

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# # Example words
# words = ["goes", "roads", "leaves"]

# # Process the words and get lemmas
# lemmas = [token.lemma_ for token in nlp(" ".join(words))]
# print(lemmas)  # Output: ['go', 'road', 'leaf']

In [22]:
def to_base_forms(s):
    try:
        # s = s.split()
        # print(s, "!!!")
        pattern = r"[^a-zA-Z']"
        s = " ".join(map(lambda x: (re.sub(pattern, "", x)).lower(), s.split()))
        # print(s)
        return (" ".join([token.lemma_ for token in nlp(s)]))
    except:
        # print(s)
        return None

vectorized_function = np.vectorize(to_base_forms)
mapped_arr = vectorized_function(df["overview"].values)

In [23]:
mapped_arr


array(["lead by woody andy 's toy live happily in his room until andy 's birthday bring buzz lightyear onto the scene afraid of lose his place in andy 's heart woody plot against buzz but when circumstance separate buzz and woody from their owner the duo eventually learn to put aside their difference",
       "when sibling judy and peter discover an enchanted board game that open the door to a magical world they unwittingly invite alan   an adult who be be trap inside the game for   year   into their living room alan 's only hope for freedom be to finish the game which prove risky as all three find themselves run from giant rhinoceros evil monkey and other terrifying creature",
       'a family wedding reignite the ancient feud between nextdoor neighbor and fishing buddy john and max meanwhile a sultry italian divorce open a restaurant at the local bait shop alarm the local who worry she will scare the fish away but she be less interested in seafood than she be in cook up a hot time wi

In [24]:
vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the corpus
tfidf_matrix = vectorizer.fit_transform([" ".join(mapped_arr)])

# Get feature names (words)
feature_names = vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix to a dense format and print
dense_matrix = tfidf_matrix.todense()

feature_names = vectorizer.get_feature_names_out()
tfidf_array = np.array(dense_matrix)
word_index = feature_names.tolist().index('friend')
word_tfidf = tfidf_array[:, word_index][0]
print("TF-IDF value for 'love':" + str(word_tfidf))
word_tfidf.shape
# print("TF-IDF Matrix:")
# print(dense_matrix)

TF-IDF value for 'love':0.13840590234633166


()

In [25]:
"to" in feature_names

False

In [26]:
tf_idf = {}
c = 0
banned_words = ['life', 'new', "film", "come", "make", "movie"]
for w in set(" ".join(mapped_arr).split()):
    try:
        if w in banned_words:
            tf_idf[w] = 0
        elif w not in feature_names:
            tf_idf[w] = 0
        else:
            word_index = feature_names.tolist().index(w)
            word_tfidf = tfidf_array[:, word_index][0].item()
            tf_idf[w] = word_tfidf
    except:
        c += 1
        print(w)
        tfidf_array[w] = 0
print(c)

0


In [27]:
tf_idf

{'motte': 5.7765401647049936e-05,
 'gromov': 2.8882700823524968e-05,
 'courtesy': 0.0003465924098822996,
 'tanztheater': 2.8882700823524968e-05,
 'fushimishi': 2.8882700823524968e-05,
 'compassionate': 0.0008664810247057491,
 'herriton': 2.8882700823524968e-05,
 'beautyeach': 2.8882700823524968e-05,
 'rodda': 2.8882700823524968e-05,
 'fellinis': 5.7765401647049936e-05,
 'bate': 0.00031770970905877464,
 'bassey': 5.7765401647049936e-05,
 'boskone': 2.8882700823524968e-05,
 'horizontal': 5.7765401647049936e-05,
 'carrie': 0.0014441350411762484,
 'freda': 0.00014441350411762484,
 'misadventure': 0.0021662025617643728,
 'axeweilding': 2.8882700823524968e-05,
 'businessso': 2.8882700823524968e-05,
 'komikliklerle': 2.8882700823524968e-05,
 'rakkyo': 2.8882700823524968e-05,
 'awkward': 0.0022239679634114226,
 'sixime': 5.7765401647049936e-05,
 'mcdougal': 2.8882700823524968e-05,
 'five': 0,
 'liotta': 0.0002888270082352497,
 'edenlike': 2.8882700823524968e-05,
 'muellerstahl': 2.888270082352

In [28]:
sorted(tf_idf.items(), key=lambda x: x[1], reverse=False)


[('five', 0),
 ("o'conor", 0),
 ("l'amor", 0),
 ('b', 0),
 ("ne'erdowell", 0),
 ('go', 0),
 ("p'ohangdong", 0),
 ("alzheimer'ssuffere", 0),
 ("'", 0),
 ("o'brien", 0),
 ("l'tranger", 0),
 ("l'homme", 0),
 ("o'briain", 0),
 ('make', 0),
 ("d'aimer", 0),
 ('of', 0),
 ('while', 0),
 ('whereas', 0),
 ('would', 0),
 ("tanaan'n", 0),
 ('six', 0),
 ('y', 0),
 ("ze'ev", 0),
 ("d'euxmme", 0),
 ('behind', 0),
 ("d'estaing", 0),
 ("d'onofrio", 0),
 ('third', 0),
 ('own', 0),
 ('i', 0),
 ("all'ispettore", 0),
 ('co', 0),
 ('movie', 0),
 ('himself', 0),
 ("problemthey're", 0),
 ('over', 0),
 ("qu'eddie", 0),
 ("d'ivan", 0),
 ("ri'chard", 0),
 ('whence', 0),
 ("l'heure", 0),
 ('hers', 0),
 ("s'attache", 0),
 ("o'day", 0),
 ("o'toole", 0),
 ("d'tre", 0),
 ('few', 0),
 ('t', 0),
 ('last', 0),
 ('onto', 0),
 ('from', 0),
 ("d'aires", 0),
 ('z', 0),
 ('mostly', 0),
 ('on', 0),
 ("didem'in", 0),
 ("o'callaghan", 0),
 ('very', 0),
 ('me', 0),
 ('there', 0),
 ("l'oppos", 0),
 ('eleven', 0),
 ('further', 0)

In [29]:
overview_keywords = list(map(lambda r: list(filter(lambda w: tf_idf[w] > 0,r.split())),mapped_arr))
df["overview_keywords"] = overview_keywords

In [30]:
"overview" in df, "overview_keywords" in df
df[['overview', 'overview_keywords']]

,overview,overview_keywords
0,"Led by Woody, Andy's toys live happily in his ...","[lead, woody, andy, toy, live, happily, room, ..."
1,When siblings Judy and Peter discover an encha...,"[sibling, judy, peter, discover, enchanted, bo..."
2,A family wedding reignites the ancient feud be...,"[family, wedding, reignite, ancient, feud, nex..."
3,"Cheated on, mistreated and stepped on, the wom...","[cheat, mistreat, step, woman, hold, breath, w..."
4,Just when George Banks has recovered from his ...,"[just, george, bank, recover, daughter, weddin..."
...,...,...
45461,Rising and falling between a man and woman.,"[rise, fall, man, woman]"
45462,An artist struggles to finish his work while a...,"[artist, struggle, finish, work, storyline, cu..."
45463,"When one of her hits goes wrong, a professiona...","[hit, wrong, professional, assassin, end, suit..."
45464,"In a small town live two brothers, one a minis...","[small, town, live, brother, minister, hunchba..."


In [31]:
df.to_csv('dataset/movies_metadata_ready_big2.csv', index=False) 

In [33]:
df

,adult,id,original_language,original_title,overview,popularity,runtime,tagline,title,vote_average,vote_count,genres,release_year,overview_keywords
0,False,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,81.0,NaN,Toy Story,7.7,5415.0,"[Animation, Comedy, Family]",1995.0,"[lead, woody, andy, toy, live, happily, room, ..."
1,False,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,104.0,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[Adventure, Fantasy, Family]",1995.0,"[sibling, judy, peter, discover, enchanted, bo..."
2,False,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,101.0,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,"[Romance, Comedy]",1995.0,"[family, wedding, reignite, ancient, feud, nex..."
3,False,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,127.0,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"[Comedy, Drama, Romance]",1995.0,"[cheat, mistreat, step, woman, hold, breath, w..."
4,False,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,106.0,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,[Comedy],1995.0,"[just, george, bank, recover, daughter, weddin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,439050,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,90.0,Rising and falling between a man and woman,Subdue,4.0,1.0,"[Drama, Family]",NaN,"[rise, fall, man, woman]"
45462,False,111109,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,360.0,NaN,Century of Birthing,9.0,3.0,[Drama],2011.0,"[artist, struggle, finish, work, storyline, cu..."
45463,False,67758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,90.0,A deadly game of wits.,Betrayal,3.8,6.0,"[Action, Drama, Thriller]",2003.0,"[hit, wrong, professional, assassin, end, suit..."
45464,False,227506,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,87.0,NaN,Satan Triumphant,0.0,0.0,[],1917.0,"[small, town, live, brother, minister, hunchba..."
